In [1]:
import pandas as pd

df = pd.read_csv("2016_Green_Taxi_Trip_Data.csv")

In [2]:
df.head()

,VendorID,lpep_pickup_datetime,Lpep_dropoff_datetime,Store_and_fwd_flag,RateCodeID,Pickup_longitude,Pickup_latitude,Dropoff_longitude,Dropoff_latitude,Passenger_count,...,MTA_tax,Tip_amount,Tolls_amount,Ehail_fee,improvement_surcharge,Total_amount,Payment_type,Trip_type,PULocationID,DOLocationID
0,2,12/09/2016 04:28:43 AM,12/09/2016 04:50:05 AM,N,1,NaN,NaN,NaN,NaN,1,...,0.5,0.0,0.0,NaN,0.3,33.3,2,1.0,256.0,123.0
1,2,12/09/2016 04:14:13 AM,12/09/2016 04:28:36 AM,N,1,NaN,NaN,NaN,NaN,5,...,0.5,0.0,0.0,NaN,0.3,10.8,2,1.0,82.0,173.0
2,2,12/09/2016 04:40:24 AM,12/09/2016 04:58:23 AM,N,1,NaN,NaN,NaN,NaN,5,...,0.5,0.0,0.0,NaN,0.3,19.8,1,1.0,82.0,236.0
3,2,12/09/2016 04:15:18 AM,12/09/2016 04:21:13 AM,N,1,NaN,NaN,NaN,NaN,1,...,0.5,0.0,0.0,NaN,0.3,7.8,2,1.0,7.0,223.0
4,2,12/09/2016 04:38:33 AM,12/09/2016 04:46:14 AM,N,1,NaN,NaN,NaN,NaN,1,...,0.5,0.0,0.0,NaN,0.3,10.8,2,1.0,7.0,129.0


In [3]:
df['Pickup_longitude'] = df['Pickup_longitude'].apply(lambda x: round(x, 5))
df['Pickup_latitude'] = df['Pickup_latitude'].apply(lambda x: round(x, 5))
df['Dropoff_longitude'] = df['Dropoff_longitude'].apply(lambda x: round(x, 5))
df['Dropoff_latitude'] = df['Dropoff_latitude'].apply(lambda x: round(x, 5))

In [4]:
import shapefile
import pandas as pd
from pyproj import Proj

def get_lati_longi(shape):
    content = []
    for sr in shape.shapeRecords():
        shape = sr.shape
        rec = sr.record
        loc_id = rec[shp_dic['LocationID']]

        x = (shape.bbox[0] + shape.bbox[2]) / 2
        y = (shape.bbox[1] + shape.bbox[3]) / 2

        p = Proj("+proj=lcc +lat_1=40.66666666666666 +lat_2=41.03333333333333 +lat_0=40.16666666666666 +lon_0=-74 +x_0=300000 +y_0=0 +datum=NAD83 +units=us-ft +no_defs ")
        x2,y2 = p(x, y, inverse=True)

        content.append((loc_id, x2, y2))

    return pd.DataFrame(content, columns=["LocationID", "longitude", "latitude"])



shape = shapefile.Reader("taxi_zones/taxi_zones.shp")

fields = shape.fields[1:]
field_name = []

for field in fields:
    field_name.append(field[0])


shp_dic = dict(zip(field_name, list(range(len(field_name)))))
all_attributes = shape.records()
shp_attr = [dict(zip(field_name, attr)) for attr in all_attributes]
df_loc = pd.DataFrame(shp_attr).join(get_lati_longi(shape).set_index("LocationID"), on="LocationID")

df_loc = df_loc.drop_duplicates(subset=['LocationID','longitude','latitude'], keep='first')
for i in range(len(df_loc)-1):
    j = i+1
    k = 1
    while j<len(df_loc) and df_loc.iloc[i]['LocationID']==df_loc.iloc[j]['LocationID']:
        df_loc.iloc[j, df_loc.columns.get_loc('LocationID')]= (df_loc.iloc[i]['LocationID'])+k
        k += 1
        j += 1
# df_loc.to_csv("taxi_zones.csv", sep=',')
df_loc.head()



,LocationID,OBJECTID,Shape_Area,Shape_Leng,borough,zone,longitude,latitude
0,1,1,0.000782,0.116357,EWR,Newark Airport,-74.171526,40.689488
1,2,2,0.004866,0.433470,Queens,Jamaica Bay,-73.822490,40.610791
2,3,3,0.000314,0.084341,Bronx,Allerton/Pelham Gardens,-73.844947,40.865745
3,4,4,0.000112,0.043567,Manhattan,Alphabet City,-73.977726,40.724137
4,5,5,0.000498,0.092146,Staten Island,Arden Heights,-74.187537,40.550665


In [5]:
template = pd.DataFrame([x for x in range(1,max(df_loc['LocationID'].tolist()))], columns=["LocationID"])

In [ ]:
template.head()

In [ ]:
df_loc.to_csv("taxi_zones.csv", sep=',')

In [6]:
df[['PULocationID', 'DOLocationID']] = df[['PULocationID', 'DOLocationID']].fillna(value=0)
df.head()

,VendorID,lpep_pickup_datetime,Lpep_dropoff_datetime,Store_and_fwd_flag,RateCodeID,Pickup_longitude,Pickup_latitude,Dropoff_longitude,Dropoff_latitude,Passenger_count,...,MTA_tax,Tip_amount,Tolls_amount,Ehail_fee,improvement_surcharge,Total_amount,Payment_type,Trip_type,PULocationID,DOLocationID
0,2,12/09/2016 04:28:43 AM,12/09/2016 04:50:05 AM,N,1,NaN,NaN,NaN,NaN,1,...,0.5,0.0,0.0,NaN,0.3,33.3,2,1.0,256.0,123.0
1,2,12/09/2016 04:14:13 AM,12/09/2016 04:28:36 AM,N,1,NaN,NaN,NaN,NaN,5,...,0.5,0.0,0.0,NaN,0.3,10.8,2,1.0,82.0,173.0
2,2,12/09/2016 04:40:24 AM,12/09/2016 04:58:23 AM,N,1,NaN,NaN,NaN,NaN,5,...,0.5,0.0,0.0,NaN,0.3,19.8,1,1.0,82.0,236.0
3,2,12/09/2016 04:15:18 AM,12/09/2016 04:21:13 AM,N,1,NaN,NaN,NaN,NaN,1,...,0.5,0.0,0.0,NaN,0.3,7.8,2,1.0,7.0,223.0
4,2,12/09/2016 04:38:33 AM,12/09/2016 04:46:14 AM,N,1,NaN,NaN,NaN,NaN,1,...,0.5,0.0,0.0,NaN,0.3,10.8,2,1.0,7.0,129.0


In [ ]:
location_df = df[(df['PULocationID'] != 0) & (df['DOLocationID'] != 0)]
actual_df = df[(df['PULocationID'] == 0) & (df['DOLocationID'] == 0)]
print(len(location_df))
print(len(actual_df))

In [ ]:
location_df['Pickup_longitude'] = location_df['PULocationID'].map(df_loc.set_index('LocationID')['longitude'])
location_df['Pickup_latitude'] = location_df['PULocationID'].map(df_loc.set_index('LocationID')['latitude'])
location_df['Dropoff_longitude'] = location_df['DOLocationID'].map(df_loc.set_index('LocationID')['longitude'])
location_df['Dropoff_latitude'] = location_df['DOLocationID'].map(df_loc.set_index('LocationID')['latitude'])
print(len(location_df))

In [ ]:
# data = actual_df.append(location_df)
# data.tail()
location_df.to_csv("loc_df.csv", sep='\t')
actual_df.to_csv("actual_sf.csv", sep='\t')

In [ ]:
location_df.head()